In [1]:
import os
import numpy as np
import pandas as pd
import altair as alt
import sys
import json

sys.path.insert(0, '..')
from agents import PsAgent
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [98]:
lista = [i for i in os.listdir('../data/models') if i not in ['.ipynb_checkpoints']]

In [99]:
lista

['2025-03-25_00-10-41.196412',
 '2025-03-25_00-08-28.328508',
 '2025-03-25_00-05-56.259962',
 '2025-03-25_00-11-39.489311',
 '2025-03-24_22-27-53.486605',
 '2025-03-24_22-19-42.734840',
 '2025-03-24_22-50-12.916953',
 '2025-03-24_22-45-16.735455',
 '2025-03-25_00-04-52.998253',
 '2025-03-25_00-04-38.335599',
 '2025-03-25_00-14-35.409127',
 '2025-03-25_00-10-31.404763',
 '2025-03-25_00-08-51.343942',
 '2025-03-25_00-15-41.049304',
 '2025-03-25_00-09-00.187272',
 '2025-03-24_22-50-27.006276',
 '2025-03-25_00-15-19.197838',
 '2025-03-25_00-12-46.956709',
 '2025-03-25_00-15-19.575090',
 '2025-03-25_00-08-19.523542',
 '2025-03-25_00-15-37.239015',
 '2025-03-25_00-05-19.590220',
 '2025-03-24_22-38-36.180681',
 '2025-03-25_00-18-47.476742',
 '2025-03-25_00-13-58.764360',
 '2025-03-25_00-09-12.127614',
 '2025-03-25_00-01-16.923349',
 '2025-03-25_00-14-03.319649',
 '2025-03-24_22-09-06.163126',
 '2025-03-25_00-09-02.348252',
 '2025-03-25_00-09-33.962757',
 '2025-03-25_00-05-09.383354',
 '2025-0

In [100]:
dados = []
for model in lista:
    path = f'../data/models/{model}'
    
    with open(f'{path}/args.json', 'r') as file:
        args = json.load(file)
        
    n_reflect = args['num_reflections']
    tao = args['tao']
    colision_flag = args['colision']
    agent = PsAgent.load(path)
    matrix = agent.h_matrix
    if colision_flag:
        for state in [0,1]:
            for colision in [0,1]:
                for T in range(0,2000):
                    observation = [state,T,colision]
                    percept = agent.percept_preprocess(observation)

                    h_vector = matrix[:, percept]
                    sum_ = np.sum(h_vector)
                    probs = h_vector/sum_

                    nao_troca = probs[0]
                    troca = probs[1]

                    _ = {
                        'model':model,
                        'reflections':args['num_reflections'],
                        'T':T,#/tao,
                        'state':state,
                        'colision_flag':colision_flag,
                        'colision':colision,
                        'troca':troca,
                        'nao_troca':nao_troca,
                    }

                    dados.append(_)
    else:
        for state in [0,1]:
            for T in range(0,2000):
                observation = [state,T]
                percept = agent.percept_preprocess(observation)

                h_vector = matrix[:, percept]
                sum_ = np.sum(h_vector)
                probs = h_vector/sum_

                nao_troca = probs[0]
                troca = probs[1]

                _ = {
                    'model':model,
                    'reflections':args['num_reflections'],
                    'T':T,#/tao,
                    'state':state,
                    'colision_flag':colision_flag,
                    'colision':np.nan,
                    'troca':troca,
                    'nao_troca':nao_troca,
                }

                dados.append(_)

# Sem colisão

In [147]:
db = pd.DataFrame(dados)
db = db[~db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','T']).reset_index(drop = True)

db['prod_nao_troca'] = db.groupby(['model','state'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state'])['P_T_troca'].cumsum().shift(1)

db_mean = db[db['T'].between(2,200)].groupby(['reflections','T','state'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,troca,P_T_troca,sum_P_T_troca
0,0,2,0,0.005239,0.005228,0.009575
1,0,2,1,0.000206,0.000206,0.000142
2,0,3,0,0.019548,0.019478,0.014803
3,0,3,1,0.000047,0.000047,0.000348
4,0,4,0,0.006070,0.004958,0.034281


In [148]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [149]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'state:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [150]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    color = 'state:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

# Com colisão

In [155]:
db = pd.DataFrame(dados)
db = db[db['colision_flag'].apply(bool)]
db = db.sort_values(['model','state','colision','T']).reset_index(drop = True)

db['prod_nao_troca'] = db.groupby(['model','state','colision'])['nao_troca'].cumprod().shift(1)
db['P_T_troca'] = db['prod_nao_troca']*db['troca']
db['sum_P_T_troca'] = db.groupby(['model','state','colision'])['P_T_troca'].cumsum().shift(1)

db_mean = db[db['T'].between(2,500)].groupby(['reflections','T','state','colision'])[
    ['troca', 'P_T_troca', 'sum_P_T_troca']
].mean().reset_index()

db_mean.head()

,reflections,T,state,colision,troca,P_T_troca,sum_P_T_troca
0,0,2,0,0.0,0.003716,0.003684,0.004413
1,0,2,0,1.0,0.010806,0.010622,0.008243
2,0,2,1,0.0,0.000634,0.000633,0.000629
3,0,2,1,1.0,0.000898,0.000884,0.001937
4,0,3,0,0.0,0.003797,0.003750,0.008097


In [156]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [157]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('P_T_troca').axis(format = '%'),
    row = 'state:N',
    column = 'colision:N',
    color = 'reflections:N'
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)

In [158]:
alt.Chart(db_mean).mark_line().encode(
    x = 'T',
    y = alt.Y('sum_P_T_troca').axis(format = '%'),
    color = 'state:N',
    row = 'colision:N',
).properties(height = 100).resolve_scale(y = 'independent')

alt.Chart(...)